In [1]:
import sys
import pandas as pd
import numpy as np
import datetime
import glob
import gc
import os

#========================================================================
# Args
#========================================================================
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month']

win_path = f'../features/4_winner/*.gz'
fname=''
# submit = pd.read_csv('../input/sample_submission.csv')
submit = []

HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from preprocessing import get_ordinal_mapping
from utils import logger_func
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())

2019-01-17 07:22:04,853 utils 396 [INFO]    [logger_func] start 


### Feature importance Gain

In [20]:
import pandas as pd
valid = pd.read_csv('../valid/0109_082_valid_lgb_lr0.01_182feats_10seed_OUT30.1834_CV3.6484994493282237_LB.csv')
# rank_cols = [col for col in valid.columns if col.count('rank')]
rank = valid[['feature'] + rank_cols].set_index('feature')
max_rank = rank.stack().reset_index().groupby(['feature'])[0].max()
min_rank = rank.stack().reset_index().groupby(['feature'])[0].min()
diff_rank = (max_rank - min_rank).sort_values()
pd.concat([rank, diff_rank], axis=1).sort_values(by=0, ascending=False)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,rank,rank_605,rank_1212,rank_1222,rank_405,rank_1128,rank_1012,rank_328,rank_2005,rank_2019,0
111_ker_new_hist_month_nunique@,152,152,150,154,154,154,134,130,155,154,25
111_ker_new_hist_month_lag_max@,59,45,51,54,63,55,63,50,57,56,18
114_mer_merchant_id_M_ID_79692349d6@,151,147,141,153,151,149,158,151,156,150,17
114_mer_merchant_id_M_ID_ee75252e6a@,169,167,168,166,167,164,166,181,169,168,17
111_ker_new_hist_month_lag_min@,101,101,114,113,101,116,102,100,114,102,16
114_mer_merchant_id_M_ID_f40ccc1312@,158,162,162,161,160,173,157,157,163,166,16
111_ker_new_hist_month_diff_mean@,78,79,78,83,74,68,78,77,81,77,15
114_mer_merchant_id_M_ID_620909e2a7@,176,173,171,170,170,163,173,172,177,175,14
114_mer_merchant_id_M_ID_48257bb851@,159,164,163,155,164,166,165,161,162,152,14
114_mer_merchant_id_M_ID_9e84cda3b1@,119,113,105,116,113,111,112,116,115,110,14


### Regression result stack to EDA

In [6]:
df = utils.read_pkl_gzip('../stack/0109_190_stack_lgb_lr0.01_182feats_1seed_OUT25.8784_CV4-103946343538847_LB.gz')
train = pd.read_csv('../input/train.csv')
df = df.merge(train, how='left', on=key)
df.to_csv('../output/0109_elo_log_regression_outlier_pred_check.csv', index=False)

### Clf result stack to EDA

In [24]:
# CLF OUTLIER EDA
df = utils.read_pkl_gzip('../stack/0106_153_outlier_classify_lgb_binary_CV0-79529175784056_179features.gz')
for col in df.columns:
    if col==key:continue
    df.rename(columns={col:f'clf_{col}'}, inplace=True)
reg = utils.read_pkl_gzip('../stack/0106_104_en_route_9seed_lgb_CV3-6493435514018864_179features.gz')
drop_cols = [col for col in reg.columns if col.count('target_')]
reg.drop(drop_cols, axis=1, inplace=True)

df = df.merge(reg, how='inner', on=[key])
df['clf_pred_mean'] = df['clf_prediction']
df.to_csv('../output/0106_13_elo_LB3689_merge_difficult_clf_outlier_9seed_CV0-7952_179features.csv', index=False)
display(df.head())

,card_id,clf_target,clf_prediction,prediction,prediction_1,prediction_2,prediction_3,prediction_4,prediction_5,prediction_6,prediction_7,target,prediction_8,pred_mean,pred_std,clf_pred_mean
0,C_ID_00007093c1,0.0,0.001822,0.051732,0.470688,-0.001880,0.036275,0.090182,0.248184,0.100091,0.335278,0.134077,0.243718,0.174919,0.158796,0.001822
1,C_ID_0001238066,NaN,0.000310,-0.252665,-0.238237,-0.241273,-0.244788,-0.238180,-0.241613,-0.240001,-0.242178,NaN,-0.245657,-0.242732,0.004521,0.000310
2,C_ID_0001506ef0,0.0,0.000791,0.186385,0.200230,0.175842,0.237972,0.200450,0.261319,0.231371,0.195033,0.884295,0.189647,0.208694,0.028299,0.000791
3,C_ID_0001793786,NaN,0.000757,-2.807433,-2.981807,-2.894369,-2.799326,-3.071587,-3.010835,-2.972943,-2.788901,NaN,-2.726459,-2.894851,0.119737,0.000757
4,C_ID_000183fdda,0.0,0.000075,-0.051847,-0.083281,-0.109405,-0.031079,-0.064012,-0.006654,-0.069058,-0.062590,-0.264433,-0.095745,-0.063741,0.031599,0.000075


In [3]:
train = utils.read_df_pkl('../input/train0*')
test = utils.read_df_pkl('../input/test0*')

100%|██████████| 3/3 [00:00<00:00, 186.05it/s]


37086.9

In [9]:
print(test.shape[0] * 0.3) # 37087
print(37087 * 3.688)
print(33 * 37087 * 0.01)

37086.9
136776.856
12238.710000000001


### これまでのSubmitのOUTLIER SCOREを確認する

In [16]:
import re
from tqdm import tqdm
from sklearn.metrics import mean_squared_error

# outlierに対するスコアを出す
train = utils.read_df_pkl('../input/train0*')
out_ids = train.loc[train.target<-30, key].values
out_val = train.loc[train.target<-30, target].values

# outlier以外に対するスコアを出す
com_ids = train.loc[train.target>-30, key].values
com_val = train.loc[train.target>-30, target].values

path_list = glob.glob('../log_submit/01*csv')
path_list = glob.glob('../submit/dir_12/*csv')
path_list = glob.glob('../submit/dir_0101_0109/*csv')
stack_list = glob.glob('../stack/*gz')

out_dict = {}
for i, path in enumerate(tqdm(path_list)):
    tmp_path = path.replace(".", '-')
    cv = re.search(r'CV([^/.]*)_LB', tmp_path).group(1)[:-2].replace('.', '-')
    lb = re.search(r'LB([^/.]*).csv', tmp_path).group(1).replace('-', '.')
    try:
        stack_path = [s for s in stack_list if s.count(cv.replace('.', '-'))][0]
    except IndexError:
        continue
    
    df_pred = utils.read_pkl_gzip(stack_path)
    
    if len(df_pred.columns)>3:
        try:
            com_pred = df_pred[df_pred[key].isin(com_ids)]['pred_mean'].values
            out_pred = df_pred[df_pred[key].isin(out_ids)]['pred_mean'].values
        except KeyError:
            com_pred = df_pred[df_pred[key].isin(com_ids)]['prediction'].values
            out_pred = df_pred[df_pred[key].isin(out_ids)]['prediction'].values
    else:
        com_pred = df_pred[df_pred[key].isin(com_ids)]['prediction'].values
        out_pred = df_pred[df_pred[key].isin(out_ids)]['prediction'].values
        
    com_score = np.sqrt(mean_squared_error(com_val, com_pred))
    try:
        out_score = np.sqrt(mean_squared_error(out_val, out_pred))
    except ValueError:
        print(out_val.shape, out_pred.shape)
        continue
    out_vals = {
        '1_CV' : cv.replace('-', '.'),
        '2_LB' : lb,
        '3_OUT': out_score,
        '4_COM': com_score,
               }
    out_dict[str(i)] = out_vals
    
result = pd.DataFrame(out_dict)
display(result.T)
sys.exit()

tmp = result.T['1_CV'][result.T['1_CV']=='3.65133231891838'].index
base = result[tmp]
comp = (result.astype('float').values - base.astype('float').values).T
df = pd.DataFrame(comp.T, columns=result.loc['1_CV', :], index=result.index).T
df.sort_values(by="3_OUT", inplace=True)
display(df)



  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 3/3 [00:00<00:00, 81.78it/s]

  0%|          | 0/153 [00:00<?, ?it/s]

  1%|          | 1/153 [00:00<00:57,  2.62it/s]

  1%|▏         | 2/153 [00:01<01:26,  1.74it/s]

  2%|▏         | 3/153 [00:01<01:17,  1.94it/s]

  3%|▎         | 4/153 [00:02<01:10,  2.13it/s]

  3%|▎         | 5/153 [00:02<01:11,  2.08it/s]

  4%|▍         | 6/153 [00:03<01:08,  2.14it/s]

  5%|▍         | 7/153 [00:03<01:22,  1.77it/s]

  5%|▌         | 8/153 [00:04<01:14,  1.94it/s]

  6%|▌         | 9/153 [00:04<01:08,  2.10it/s]

  7%|▋         | 10/153 [00:05<01:04,  2.23it/s]

  7%|▋         | 11/153 [00:05<01:06,  2.14it/s]

  8%|▊         | 12/153 [00:05<01:01,  2.29it/s]

  8%|▊         | 13/153 [00:06<01:15,  1.85it/s]

  9%|▉         | 14/153 [00:07<01:05,  2.13it/s]

 10%|▉         | 15/153 [00:07<01:17,  1.77it/s]

 10%|█         | 16/153 [00:08<01:09,  1.96it/s]

 11%|█         | 17/153 [00:08<01:09,  1.95it/s]

 12%|█▏        | 18/153 [00:09

,1_CV,2_LB,3_OUT,4_COM
0,3.6507835250103,,30.2089,2.07016
1,3.6501559181335,,30.182,2.06686
2,3.64975098605533,,30.2381,2.08232
3,3.65216600497249,,30.2672,2.06801
4,3.65136990764986,,30.1931,2.07421
5,3.6537984246267,,30.242,2.06432
6,3.6492050017024,,30.1755,2.06535
7,3.64958482363406,,30.1693,2.0784
8,3.6534990502266,,30.2445,2.06695
9,3.65078116541322,,30.2065,2.07322


SystemExit: 

In [18]:
result.T.sort_values(by='3_OUT')

,1_CV,2_LB,3_OUT,4_COM
54,1.54708702804528,,0.465429,1.86617
118,1.54708702804528,,0.465429,1.86617
86,3.6505326548175,,30.1623,2.08244
119,3.6505326548175,,30.1623,2.08244
59,3.65013200733974,,30.1637,2.07995
79,3.64958482363406,,30.1693,2.0784
7,3.64958482363406,,30.1693,2.0784
34,3.6490668656652,,30.1717,2.0662
74,3.6496609119658,,30.1737,2.07628
11,3.64995553434516,,30.1741,2.07818


### Blend Corr

In [2]:
import re
import sys
import glob
import pandas as pd
pd.set_option("max_rows", 200)
path_list = glob.glob('../log_submit/*.csv')

tmp_list = []
lb_list = []
path_list = list(set(path_list))
for path in path_list:
    if not(path.count('CV')) or not(path.count('LB')):
        continue
    print(path)
    tmp = pd.read_csv(path)
    tmp_path = path.replace(".", '-')
    cv = re.search(r'CV([^/.]*)_LB', tmp_path).group(1)[:-2].replace('-', '.')
    lb = re.search(r'LB([^/.]*).csv', tmp_path).group(1).replace('-', '.')
    if len(lb)==0 or float(cv)<3:
        continue
    tmp.rename(columns={'target':f"CV{cv[:9]}_LB{lb}"}, inplace=True)
    tmp.set_index('card_id', inplace=True)
    tmp_list.append(tmp.copy())
    lb_list.append(lb)

df = pd.concat(tmp_list, axis=1)
df_corr = df.corr(method='pearson')
df_corr.columns = [col[-5:] for col in df.columns]
df_corr['LB'] = lb_list
df_corr = df_corr.sort_values(by='LB')
df_corr.sort_index(ascending=True, axis=1, inplace=True)
df_corr.head()

../log_submit/1204_085_submit_lgb_rate0.1_56features_CV3.7726010730653825_LB.csv
../log_submit/0103_004_submit_lgb_rate0.01_156features_1seed_CV3.651246362821579_LB3.690.csv
../log_submit/0114_004_submit_lgb_lr0.01_177feats_10seed_31leaves_iter1312_OUT30.2179_CV3.6505372648747247_LB3.688.csv
../log_submit/1228_003_submit_lgb_rate0.01_143features_CV3.6523085700161366_LB.csv
../log_submit/0108_102_submit_lgb_lr0.005_178feats_1seed_OUT30.4728_CV3.6554034556948864_LB3.697.csv
../log_submit/0106_084_submit_lgb_rate0.01_154features_CV1.5538050162614394_LB.csv
../log_submit/0114_235_submit_lgb_lr0.01_274feats_1seed_48leaves_iter1055_OUT30.4463_CV3.668079056540345_LB3.721.csv
../log_submit/1228_112_submit_lgb_rate0.01_151features_1seed_CV3.6519799986945727_LB.csv
../log_submit/0103_155_submit_lgb_rate0.01_178features_9seed_CV3.650610498430334_LB3.689.csv
../log_submit/1229_203_submit_lgb_rate0.005_124features_1seed_CV3.651480922647848_LB.csv
../log_submit/0110_083_submit_lgb_lr0.01_160feats_1s

,3.687,3.687,3.687,3.688,3.688,3.688,3.688,3.688,3.688,3.688,...,3.695,3.696,3.697,3.698,3.700,3.703,3.708,3.721,3.728,LB
CV3.6492564_LB3.687,1.000000,0.993585,0.978876,0.990994,0.990051,0.991288,0.992202,0.996731,0.985249,0.995864,...,0.977195,0.982037,0.980061,0.975640,0.954837,0.960331,0.947885,0.925879,0.920408,3.687
CV3.3502559_LB3.687,0.978876,0.982717,1.000000,0.984025,0.984439,0.982721,0.983080,0.980398,0.991392,0.981237,...,0.968689,0.973020,0.980936,0.976960,0.945933,0.964115,0.941623,0.923732,0.907894,3.687
CV3.6490461_LB3.687,0.993585,1.000000,0.982717,0.985993,0.985083,0.986799,0.986996,0.992187,0.980574,0.995591,...,0.973651,0.977858,0.982229,0.977324,0.958902,0.967101,0.943957,0.923444,0.922672,3.687
CV3.6488459_LB3.688,0.992272,0.987738,0.982849,0.996389,0.995401,0.998336,0.997974,0.995066,0.990024,0.988638,...,0.983784,0.989185,0.985136,0.978447,0.951984,0.961078,0.953508,0.926134,0.914547,3.688
CV3.6505372_LB3.688,0.994273,0.994227,0.980166,0.987664,0.986913,0.986281,0.988455,0.991780,0.981818,0.998187,...,0.974484,0.977576,0.980109,0.977919,0.953766,0.963470,0.943727,0.925664,0.920781,3.688
CV3.6493435_LB3.688,0.990051,0.985083,0.984439,0.998884,1.000000,0.995443,0.997106,0.991809,0.991801,0.988308,...,0.979045,0.985479,0.984466,0.982625,0.949733,0.963936,0.953789,0.925495,0.913712,3.688
CV3.6496146_LB3.688,0.990994,0.985993,0.984025,1.000000,0.998884,0.994880,0.998079,0.992783,0.991163,0.989225,...,0.979980,0.986367,0.985654,0.981819,0.950885,0.962511,0.953257,0.925769,0.913647,3.688
CV3.6474992_LB3.688,0.985249,0.980574,0.991392,0.991163,0.991801,0.989879,0.990096,0.987050,1.000000,0.983076,...,0.974314,0.979932,0.978645,0.975101,0.944193,0.959964,0.945627,0.927324,0.909392,3.688
CV3.6497211_LB3.688,0.995864,0.995591,0.981237,0.989225,0.988308,0.987716,0.989974,0.993351,0.983076,1.000000,...,0.975969,0.979011,0.981295,0.979258,0.954514,0.963945,0.944928,0.925105,0.920174,3.688
CV3.6488796_LB3.688,0.996731,0.992236,0.980446,0.992807,0.991831,0.994151,0.994443,0.999789,0.987041,0.993379,...,0.979331,0.984980,0.982276,0.976477,0.952478,0.960724,0.950522,0.927120,0.918211,3.688


In [3]:
pd.set_option('display.max_rows', 60)
# pd.set_option("max_rows", 200)
df_corr.iloc[:, :25]

,3.687,3.687,3.687,3.688,3.688,3.688,3.688,3.688,3.688,3.688,...,3.689,3.689,3.689,3.689,3.690,3.690,3.690,3.690,3.690,3.690
CV3.6492564_LB3.687,1.000000,0.993585,0.978876,0.990994,0.990051,0.991288,0.992202,0.996731,0.985249,0.995864,...,0.991024,0.984812,0.990445,0.985539,0.986061,0.986949,0.985511,0.988997,0.984322,0.983305
CV3.3502559_LB3.687,0.978876,0.982717,1.000000,0.984025,0.984439,0.982721,0.983080,0.980398,0.991392,0.981237,...,0.982142,0.986227,0.981604,0.975278,0.988249,0.985815,0.987941,0.983580,0.988020,0.986816
CV3.6490461_LB3.687,0.993585,1.000000,0.982717,0.985993,0.985083,0.986799,0.986996,0.992187,0.980574,0.995591,...,0.985597,0.986793,0.985314,0.990357,0.988730,0.991791,0.988414,0.984155,0.988692,0.987378
CV3.6488459_LB3.688,0.992272,0.987738,0.982849,0.996389,0.995401,0.998336,0.997974,0.995066,0.990024,0.988638,...,0.996234,0.989427,0.996058,0.980108,0.990711,0.993463,0.990156,0.994379,0.989091,0.988185
CV3.6505372_LB3.688,0.994273,0.994227,0.980166,0.987664,0.986913,0.986281,0.988455,0.991780,0.981818,0.998187,...,0.987162,0.986185,0.986787,0.985952,0.987945,0.986272,0.987478,0.986832,0.987133,0.985883
CV3.6493435_LB3.688,0.990051,0.985083,0.984439,0.998884,1.000000,0.995443,0.997106,0.991809,0.991801,0.988308,...,0.995827,0.993342,0.995107,0.977979,0.994938,0.988979,0.994381,0.998830,0.993406,0.992477
CV3.6496146_LB3.688,0.990994,0.985993,0.984025,1.000000,0.998884,0.994880,0.998079,0.992783,0.991163,0.989225,...,0.996727,0.992615,0.996025,0.978849,0.994046,0.989897,0.993468,0.997794,0.992494,0.991522
CV3.6474992_LB3.688,0.985249,0.980574,0.991392,0.991163,0.991801,0.989879,0.990096,0.987050,1.000000,0.983076,...,0.989566,0.986280,0.988836,0.973673,0.987490,0.984024,0.986934,0.990840,0.985752,0.984972
CV3.6497211_LB3.688,0.995864,0.995591,0.981237,0.989225,0.988308,0.987716,0.989974,0.993351,0.983076,1.000000,...,0.988705,0.987322,0.988278,0.987028,0.989128,0.987531,0.988625,0.988209,0.988345,0.987202
CV3.6488796_LB3.688,0.996731,0.992236,0.980446,0.992807,0.991831,0.994151,0.994443,0.999789,0.987041,0.993379,...,0.992795,0.986349,0.992651,0.984247,0.987516,0.989727,0.986981,0.990726,0.985823,0.984861
